![image-3.png](https://github.com/cafawo/Derivatives/blob/main/figures/derivatives_header.png?raw=1)

# Derivatives

[**Prof. Dr. Fabian Woebbeking**](https://woebbeking.info/)</br>
Assistant Professor of Financial Economics</br>

## Case 3: Fixed Income

### Part 1: Interest rates

The following 3 zero bonds are given:

| Maturity | Price | Redemption |
|:--------:|:-----:|:----------:|
|    1Y    | 95.24 |     100    |
|    2Y    | 89.00 |     100    |
|    3Y    | 81.63 |     100    |

(Table generated with [TablesGenerator](https://www.tablesgenerator.com/markdown_tables))

#### Task: Calculate spot and forward rates.

#### Task: Price a three year 10% bond. Calculate the YTM and explain the relationship of the spot rates and the YTM.

#### Task: Solve analytically and calculate the par coupon rate for a three year bond.

#### Task: Evaluate with the same curve, a three year floater against the one year rate (bond base). ... For the fans :)

### Part 2: Asset-/ liability-swap

#### Task: Calculate the level of the floating debt (liability swap).

Power has issued a five year bond with a coupon of 2% and a price of 100.50. The five year IRS is quoted 2,00 – 2,05 against six month EURIBOR.

#### Task: Calculate the asset swap.

The Investor has bought a five year AA Bond with a coupon of 3% for 101,00. The 5 Y IRS quotes 2,00 -2,05 against 6M Euribor.

### Part 3: Single curve

The following swap rates are given:

| Maturity | 12M Euribor | Eonia |
|:--------:|:-----------:|:-----:|
|    1Y    |    4.00%    | 3.30% |
|    2Y    |    5.00%    | 4.50% |
|    3Y    |    6.00%    | 5.30% |

(Tables generated with [TablesGenerator](https://www.tablesgenerator.com/markdown_tables))

#### Task: Bootstrap discount factors and spot rates using the single curve approach.

#### Task: Price a three year swap against the one year rate using the single curve approach.

#### Task: Price the forward swap in one for two. Show that the result is correct and compare it to the zero rate in one for two using the single curve approach.

#### Task: Mark to market a swap, 8% against 12M Euribor with a remaining maturity of 3 years and EUR 100mln notional.

### Part 4: Multi curve

Consider a three year swap against the one year rate using the multi curve approach. The following swap rates are given:

| Maturity | 12M Euribor | Eonia |
|:--------:|:-----------:|:-----:|
|    1Y    |    4.00%    | 3.30% |
|    2Y    |    5.00%    | 4.50% |
|    3Y    |    6.00%    | 5.30% |

(Tables generated with [TablesGenerator](https://www.tablesgenerator.com/markdown_tables))

#### Task: Calculate the spot rates.

#### Task: Calculate the implied forward rates.

(Hint: the one year spot rate for the swap cash flow is taken from the single curve approach.)


In [2]:
def swap_bootstrap(swap_rates):
    discount_factors = []
    for s in swap_rates:
        # See Slide 61
        DF_T = (1 - s * sum(discount_factors)) / (1 + s)
        discount_factors.append(DF_T)
        return [discount_factors]



"""Calculate implied forward rates (multi curve approach)
"""
def multi_curve(swap_rates, eonia_rates):
    # bootstrap discount factors from eonia rates
    discount_factors = swap_bootstrap(eonia_rates)[0]
    # setup
    SDF = 0 # sum of discount factors
    CD = 0  # discounted fixed leg until T
    LD = 0  # discounted float leg until T-1

    implied_forwards = []
    for C_T, DF_T in zip(swap_rates, discount_factors):
        SDF += DF_T  # add up discount factors
        CD = C_T * SDF  # C_T is an annuity
        L = (CD - LD) / DF_T  # calculate implied forward rate (see equation above)
        implied_forwards.append(round(L,4))  # round and save L
        LD += L * DF_T  # sum up discounted L (used in next iteration)

    return implied_forwards

print(f"Implied forward rates: {multi_curve([0.04, 0.05, 0.06], [0.033, 0.045, 0.053])}")

Implied forward rates: [0.04]


#### Task: Calculate the forward swap in one for two.


### Part 5: Interest rate parity / FX forward

#### Task: Calculate the one year (365 days) FX forward using covered interest rate parity.
* Spot = 1.3193
* USD interest (IBOR) = 1.05310%
* EUR interest (IBOR) = 1.4375%

In [1]:
# FX outright
r_EUR = 1.43750 / 100
r_USD = 1.05310 / 100
EURUSD = 1.3193
T_act = 365

FX_Fwd = EURUSD * (1 + r_USD * T_act / 360)/(1 + r_EUR * T_act / 360)

print(f"EURUSD forward =  {FX_Fwd:.4f}")
print(f"EURUSD swap  = {FX_Fwd - EURUSD:.4f}")

EURUSD forward =  1.3142
EURUSD swap    = -0.0051


#### Task: Mark to market the following cross currency swap.

The term structure of interest rates is flat at 6% in USD and 8% in AUD (continuous rates). The current exchange rate is 0.64 USD per AUD. You pay 7% in AUD and receive 3% in USD, with principals being USD 15mln and AUD 25mln. Payments are exchanged annually with 3 payments left (3 years to maturity).

In [3]:
import numpy as np

# 参数设置
r_usd = 0.06  # USD 利率 (continuous)
r_aud = 0.08  # AUD 利率 (continuous)
spot_fx = 0.64  # USD per 1 AUD

notional_usd = 15_000_000
notional_aud = 25_000_000

usd_coupon = 0.03 * notional_usd
aud_coupon = 0.07 * notional_aud

years = [1, 2, 3]

# 折现函数（连续复利）
def discount(rate, t):
    return np.exp(-rate * t)

# 折现后的现金流现值
pv_usd = sum(usd_coupon * discount(r_usd, t) for t in years) + notional_usd * discount(r_usd, 3)
pv_aud = sum(aud_coupon * discount(r_aud, t) for t in years) + notional_aud * discount(r_aud, 3)

# 将AUD现值换算为USD
pv_aud_in_usd = pv_aud * spot_fx

# 市值 = 收到的现值（USD） - 支付的现值（折算成USD）
mtm = pv_usd - pv_aud_in_usd

# 输出结果
print(f"Present value of USD leg:   {pv_usd:,.2f} USD")
print(f"Present value of AUD leg:   {pv_aud_in_usd:,.2f} USD (converted)")
print(f"Mark-to-market value:       {mtm:,.2f} USD")


Present value of USD leg:   13,727,833.00 USD
Present value of AUD leg:   15,455,360.33 USD (converted)
Mark-to-market value:       -1,727,527.33 USD
